In [2]:
import sys
sys.path.append('/media/data/hungnt/work/SourceCode/Brain_DNA/src/')

import os
import numpy as np
import pandas as pd
import config
import utils
import pickle
import argparse
import train_impf_MLP

from utils import impf_make_ndarray_from_csv, get_int_label
from Model import Impf_DNAMLP
from Dataset import CNS
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from tqdm import tqdm

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--save', type = str, default = 'no_save')
    parser.add_argument('--impf_alg', type = str, default =  'lr')
    args, _ = parser.parse_known_args()
    return args

def get_impf_alg(alg):
    switcher = {
        'mlp': 'MLP',
        'rf': 'RF',
        'lr': 'LR',
        'xgb': 'XGB'  
    }
    
    return switcher.get(alg.lower(), None)

if __name__ == "__main__":
    args = parse_args()
    save = args.save
    impf_alg = get_impf_alg(args.impf_alg)
    clf_cfg = config.classifier_config
    impf_cfg = config.impf_config
    
    groups = utils.positive_groups
    folds = utils.inner_folds
    
    MLP_BEST_STATES_DIR = impf_cfg['IMPORTANT_FEATURES_DIR']
    
    for group in tqdm(groups, desc = 'Groups: ', position = 0):
        for fold in tqdm(folds, desc = 'Fols: ', position = 1):
            outer_fold = f'{fold.split(".")[0]}.0'
            with open(os.path.join(impf_cfg['IMPORTANT_FEATURES_DIR'], impf_alg, group, f'{outer_fold}_combined.pkl'), 'rb') as file:
                impf = pickle.load(file)    
            tqdm.write(f'[MLP] | {group} - {fold} - {impf_alg} ({len(impf)})')                        
            train_features, train_labels, val_features, val_labels = impf_make_ndarray_from_csv(group, fold, impf)
            test_features, test_labels = impf_make_ndarray_from_csv(group, outer_fold, impf, mode = 'test')
            # Encode the labels
            train_labels_int = np.array([get_int_label(label, group) for label in train_labels])
            val_labels_int = np.array([get_int_label(label, group) for label in val_labels])
            test_labels_int = np.array([get_int_label(label, group) for label in test_labels])
            
            # Create datasets and Dataloaders
            train_dataset = CNS(train_features, train_labels_int, mode = 'train')
            val_dataset = CNS(val_features, val_labels_int, mode = 'val')
            test_dataset = CNS(test_features, test_labels_int, mode = 'val')
            train_loader = DataLoader(train_dataset, batch_size = impf_cfg['mlp_train_batch_size'], shuffle = True)
            val_loader = DataLoader(val_dataset, batch_size = impf_cfg['mlp_val_batch_size'], shuffle = False)
            test_loader = DataLoader(test_dataset, batch_size = impf_cfg['mlp_val_batch_size'], shuffle = False)

            # Init model object
            in_features = len(impf)
            model = Impf_DNAMLP(in_features, impf_cfg['n_classes'])
            if impf_cfg['MLP_FIRST_TIME'] == False:
                # Load model based on fold
                BEST_STATE_PATH = os.path.join(impf_cfg['MLP_BEST_STATES_DIR'], group, f'{fold}.pth')
                model.load_state_dict(torch.load(BEST_STATE_PATH))
                
            # Define training and validating hyperparams
            criterion = CrossEntropyLoss(weight=None)
            optimizer = Adam(model.parameters(), lr = impf_cfg['mlp_lr'], weight_decay = impf_cfg['mlp_weight_decay'])
            if save == 'save':
                tqdm.write('Running in save mode')
                best_accs = train_impf_MLP.impf_run(group, fold, train_loader, val_loader, test_loader, model, criterion, optimizer, impf_cfg, save)
            else:
                tqdm.write('Running in no save mode')
                best_accs = train_impf_MLP.impf_run(group, fold, train_loader, val_loader, test_loader, model, criterion, optimizer, impf_cfg, save)

                                              
Groups:   0%|          | 0/13 [00:00<?, ?it/s]

[MLP] | Embryonal - 1.1 - LR (360)


                                              
Groups:   0%|          | 0/13 [00:04<?, ?it/s]




Running in no save mode


















                                              
                                         

Groups:   0%|          | 0/13 [00:04<?, ?it/s]
                                              
                                         

Groups:   0%|          | 0/13 [00:04<?, ?it/s]
                                              
                                         

Groups:   0%|          | 0/13 [00:04<?, ?it/s]
                                              
                                         

Groups:   0%|          | 0/13 [00:04<?, ?it/s]
                                              
                                         

Groups:   0%|          | 0/13 [00:04<?, ?it/s]





...

{class_name.upper()}] - {fold} - {epoch}/{n_epochs}
train_loss: 0.33056 | train_acc: 88.095 | train_precision: 1.000 | train_recall: 0.185 | train_auc: 0.633
val_loss: 0.31305 | val_acc: 90.090 | val_precision: 1.000000 | val_recall: 0.353 | val_auc: 0.676
test_loss: 0.33745 | test_acc: 88.889 | test_precision: 1.000000 | test_recall: 0.370 | test_auc: 0.685
---------------------------------------------


















                                              
                                                 

Groups:   0%|          | 0/13 [01:23<?, ?it/s]         
                                              
                                                 

Groups:   0%|          | 0/13 [01:23<?, ?it/s]         
                                              
                                                 

Groups:   0%|          | 0/13 [01:23<?, ?it/s]         
                                              
                                                 

Groups:   0%|          | 0/13 [01:23<?, ?it/s]         
                                              
                                                 

Groups:   0%|          | 0/13 [01:23<?, ?it/s]         





{class_name.upper()}] - {fold} - {epoch}/{n_epochs}
train_loss: 0.32108 | train_acc: 89.177 | train_precision: 1.000 | train_recall: 0.177 | train_auc: 0.667
val_loss: 0.30411 | val_acc: 90.090 | val_precision: 1.000000 | val_recall: 0.353 | val_auc: 0.676
test_loss: 0.33198 | test_acc: 86.928 | test_precision: 1.000000 | test_recall: 0.259 | test_auc: 0.630
---------------------------------------------


















                                              
                                                 

Groups:   0%|          | 0/13 [01:23<?, ?it/s]         
                                              
                                                 

Groups:   0%|          | 0/13 [01:23<?, ?it/s]         
                                              
                                                 

Groups:   0%|          | 0/13 [01:23<?, ?it/s]         
                                              
                                                 

Groups:   0%|          | 0/13 [01:23<?, ?it/s]         
                                              
                                                 

Groups:   0%|          | 0/13 [01:23<?, ?it/s]         





{class_name.upper()}] - {fold} - {epoch}/{n_epochs}
train_loss: 0.31182 | train_acc: 90.693 | train_precision: 1.000 | train_recall: 0.170 | train_auc: 0.713
val_loss: 0.29544 | val_acc: 90.090 | val_precision: 1.000000 | val_recall: 0.353 | val_auc: 0.676
test_loss: 0.32159 | test_acc: 87.582 | test_precision: 1.000000 | test_recall: 0.296 | test_auc: 0.648
---------------------------------------------
